In [ ]:
!pip install -qU \
    openai==0.27.7 \
    pinecone-client==3.1.0 \
    pinecone-datasets==0.7.0 \
    tqdm \
    pinecone-notebooks==0.1.1

In [ ]:
import pandas as pd
PINECONE_API_KEY = ''# find this in env file


## Upsert campaign-index

In [ ]:
campaigns = pd.read_csv('../../../data/campaigns.csv')
campaigns.rename(columns={'Unnamed: 0': 'campaignID'}, inplace=True)
campaigns.head()

In [17]:
from pipecone_connection import create_vector_emb

campaigns['vector'] = campaigns['label'].apply(lambda x: create_vector_emb(x))
campaigns.head()

,campaignID,label,link,img,vector
0,0,"Khai trương chi nhánh 221, 222 - Cần Thơ, Đồng...",https://hasaki.vn/campaign/khai-truong-chi-nha...,https://media.hcdn.vn/hsk/campaign/640x240-172...,"[-0.06392963230609894, 0.11033326387405396, -0..."
1,1,"Khai trương chi nhánh 223, 224 - Hội An, TP.HCM",https://hasaki.vn/campaign/khai-truong-chi-nha...,https://media.hcdn.vn/hsk/campaign/640x240mobi...,"[0.009469451382756233, 0.24539247155189514, -0..."
2,2,Sale lương về - Deal ngập tràn,https://hasaki.vn/campaign/best-seller,https://media.hcdn.vn/hsk/campaign/640x240---2...,"[-0.008002284914255142, 0.26300203800201416, -..."
3,3,[HOT] Ưu đãi trải nghiệm làm đẹp đến hơn 60%,https://hasaki.vn/campaign/trai-nghiem-dich-vu,https://media.hcdn.vn/hsk/campaign/wap16981146...,"[-0.06252358853816986, 0.3143085539340973, -0...."
4,4,Unilever Nâng Niu Nét Đẹp Toàn Diện,https://hasaki.vn/campaign/unilever-khuyen-mai...,https://media.hcdn.vn/hsk/campaign/UnileverSIS...,"[-0.32031115889549255, 0.60589200258255, -0.34..."


In [22]:
from pipecone_connection import PineConeDB
# Initialize Pinecone

pc = PineConeDB(PINECONE_API_KEY)
cam_index = pc.create_index(index_name='campaign-index', dimension=len(campaigns['vector'][0]))


In [23]:
# Prepare and upsert data
campaigns_list = [{"id": str(row['campaignID']), 
         "values": row['vector'],
         "metadata": {'label': row['label'], 'link': row['link'], 'img': row['img']}} for _, row in campaigns.iterrows()]
cam_index.upsert(vectors=campaigns_list)

# Optional: Deinitialize Pinecone
# pinecone.deinit()

{'upserted_count': 13}

In [ ]:
# ref: https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/gen-qa-openai.ipynb#scrollTo=KPXw1sEubwoF

## Upsert support-index

In [39]:
support = pd.read_csv('../../../data/supports.csv')
# support = support.drop(columns=['Unnamed: 0'], axis=1)
support.rename(columns={'Unnamed: 0': 'supportID'}, inplace=True)

support.head()

,supportID,title,link,content
0,0,"<div class=""text_logo"">Tài khoản</div>",https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...
1,1,"<div class=""text_logo"">Đặt hàng</div>",https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Đặt hàng tại Hasaki | Hasaki.vn Gửi yêu cầu | ...
2,2,"<div class=""text_logo"">Quy cách đóng gói</div>",https://hotro.hasaki.vn/quy-cach-dong-goi.html,Quy cách đóng gói hàng tại Hasaki | Hasaki.vn ...
3,3,"<div class=""text_logo"">Vận chuyển 2H</div>",https://hotro.hasaki.vn/van-chuyen-2h.html,Vận chuyển 2H tại Hasaki | Hasaki.vn Gửi yêu c...
4,4,"<div class=""text_logo"">Phí vận chuyển</div>",https://hotro.hasaki.vn/phi-van-chuyen.html,Phí vận chuyển tại Hasaki | Hasaki.vn Gửi yêu ...


In [40]:
from pipecone_connection import create_vector_emb
support['vector'] = support['content'].apply(lambda x: create_vector_emb(x))
support.head()

,supportID,title,link,content,vector
0,0,"<div class=""text_logo"">Tài khoản</div>",https://hotro.hasaki.vn/tai-khoan.html,Tài khoản Hasaki | Hasaki.vn Gửi yêu cầu | Đăn...,"[-0.13323213160037994, 0.36486655473709106, -0..."
1,1,"<div class=""text_logo"">Đặt hàng</div>",https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Đặt hàng tại Hasaki | Hasaki.vn Gửi yêu cầu | ...,"[-0.12891894578933716, 0.3350728154182434, -0...."
2,2,"<div class=""text_logo"">Quy cách đóng gói</div>",https://hotro.hasaki.vn/quy-cach-dong-goi.html,Quy cách đóng gói hàng tại Hasaki | Hasaki.vn ...,"[-0.15026508271694183, 0.3293699622154236, -0...."
3,3,"<div class=""text_logo"">Vận chuyển 2H</div>",https://hotro.hasaki.vn/van-chuyen-2h.html,Vận chuyển 2H tại Hasaki | Hasaki.vn Gửi yêu c...,"[-0.15769746899604797, 0.3137083649635315, -0...."
4,4,"<div class=""text_logo"">Phí vận chuyển</div>",https://hotro.hasaki.vn/phi-van-chuyen.html,Phí vận chuyển tại Hasaki | Hasaki.vn Gửi yêu ...,"[-0.14706957340240479, 0.3330081105232239, -0...."


In [41]:
len(support['vector'][0])

768

In [28]:
from pipecone_connection import PineConeDB

pc = PineConeDB(PINECONE_API_KEY)
pindex = pc.create_index(index_name='support-index', dimension=len(support['vector'][0]))

In [42]:
# Prepare and upsert data
support_list = [{"id": str(row['supportID']), 
         "values": row['vector'],
         "metadata": {'title': row['title'], 'link': row['link'], 'content': row['content']}} for _, row in support.iterrows()]

pindex.upsert(vectors=support_list)
# # Function to chunk data
# def chunk_data(data, batch_size=100):
#     """Splits data into smaller chunks for upserting."""
#     for i in range(0, len(data), batch_size):
#         yield data[i:i + batch_size]

# # Upsert in batches
# for batch in chunk_data(product_link_list, batch_size=100):
#     pindex.upsert(vectors=batch)
 

{'upserted_count': 27}

## Upsert category-index

In [30]:
categories = pd.read_csv('../../../data/categories.csv')
categories.rename(columns={'Unnamed: 0': 'categoryID'}, inplace=True)
categories.head()

,categoryID,link,img,cat
0,0,https://hasaki.vn/danh-muc/trang-diem-moi-c24....,https://media.hcdn.vn/catalog/category/c24-tra...,Trang Điểm Môi
1,1,https://hasaki.vn/danh-muc/mat-na-c30.html,https://media.hcdn.vn/catalog/category/30_1_im...,Mặt Nạ
2,2,https://hasaki.vn/danh-muc/trang-diem-mat-c52....,https://media.hcdn.vn/catalog/category/c52-tra...,Trang Điểm Mặt
3,3,https://hasaki.vn/danh-muc/sua-rua-mat-c19.html,https://media.hcdn.vn/catalog/category/19_3_im...,Sữa Rửa Mặt
4,4,https://hasaki.vn/danh-muc/trang-diem-vung-mat...,https://media.hcdn.vn/catalog/category/c50-tra...,Trang Điểm Mắt


In [31]:
from pipecone_connection import create_vector_emb
categories['vector'] = categories['cat'].apply(lambda x: create_vector_emb(x))
categories.head()

,categoryID,link,img,cat,vector
0,0,https://hasaki.vn/danh-muc/trang-diem-moi-c24....,https://media.hcdn.vn/catalog/category/c24-tra...,Trang Điểm Môi,"[-0.3737962245941162, 0.24235257506370544, -0...."
1,1,https://hasaki.vn/danh-muc/mat-na-c30.html,https://media.hcdn.vn/catalog/category/30_1_im...,Mặt Nạ,"[-0.43052226305007935, 0.05163513496518135, -0..."
2,2,https://hasaki.vn/danh-muc/trang-diem-mat-c52....,https://media.hcdn.vn/catalog/category/c52-tra...,Trang Điểm Mặt,"[-0.1801210343837738, -0.03265438973903656, -0..."
3,3,https://hasaki.vn/danh-muc/sua-rua-mat-c19.html,https://media.hcdn.vn/catalog/category/19_3_im...,Sữa Rửa Mặt,"[-0.40071868896484375, -0.004505708813667297, ..."
4,4,https://hasaki.vn/danh-muc/trang-diem-vung-mat...,https://media.hcdn.vn/catalog/category/c50-tra...,Trang Điểm Mắt,"[-0.4118107855319977, 0.4342288374900818, -0.2..."


In [32]:
from pipecone_connection import PineConeDB

# Create Index 
pc = PineConeDB(PINECONE_API_KEY)
cindex = pc.create_index(index_name='category-index', dimension=len(categories['vector'][0]))


In [33]:
# Prepare and upsert data
category_list = [{"id": str(row['categoryID']), 
         "values": row['vector'],
         "metadata": {'link': row['link'], 'img': row['img'], 'cat': row['cat']}} for _, row in categories.iterrows()]
cindex.upsert(vectors=category_list)


{'upserted_count': 15}

## Upsert comment-index

In [34]:
import pandas as pd
comments = pd.read_csv('../../../data/comments.csv')
comments.rename(columns={'Unnamed: 0': 'commentID'}, inplace=True)
comments.head()

,commentID,Name,Time,Description,Comment,Images
0,0,Bùi Ngọc Hà Linh,14: 56 | 14/08/2023,Son Dưỡng Môi DHC Không Màu Hỗ Trợ Giảm Thâm M...,Cây này cây thứ 3 rồi vẫn chưa có dấu hiệu dừn...,[]
1,1,Như Phương,16: 51 | 31/10/2023,Son Dưỡng Môi DHC Không Màu Hỗ Trợ Giảm Thâm M...,"Cây son dưỡng mình thích nhất từ trc tới giờ, ...",['https://media.hcdn.vn/rating/169874591132980...
2,2,Phạm Thị Tuyết,19: 48 | 20/01/2022,Son Dưỡng Môi DHC Không Màu Hỗ Trợ Giảm Thâm M...,Mới từ đầu sẽ thấy mùi son hơi khó ngửi nhưng ...,[]
3,3,Minh,00: 35 | 18/11/2022,Son Dưỡng Môi DHC Không Màu Hỗ Trợ Giảm Thâm M...,"Giá cả hợp lý, giao hàng nhanh. Sản phẩm không...",[]
4,4,Hien,17: 42 | 24/08/2024,Son Dưỡng Môi DHC Không Màu Hỗ Trợ Giảm Thâm M...,"Xài mướt, cảm giác nhẹ môi chứ k nặng dàyooi n...",[]


In [ ]:
from pipecone_connection import create_vector_emb
comments['vector'] = comments['Description'].apply(lambda x: create_vector_emb(x))
comments.head()

In [ ]:
from pipecone_connection import PineConeDB

# Create Index 
pc = PineConeDB(PINECONE_API_KEY)
cmt_index = pc.create_index(index_name='comment-index', dimension=len(comments['vector'][0]))


In [ ]:
# Prepare and upsert data
comment_list = [{"id": str(row['commentID']), 
         "values": row['vector'],
         "metadata": {'comment_user': row['Name'], 'comment_at': row['Time'], 'p_comment': row['Description'],
                      'comment': row['Comment']}} for _, row in comments.iterrows()]
cmt_index.upsert(vectors=comment_list)
